In [1]:
from pulp import *
import utils

In [2]:
INSTANCE_NUMBER = 5
ALL_SOLUTIONS = False
ROTATION = True

file_path = f"../instances/ins-{INSTANCE_NUMBER}.txt"
w, n, dims = utils.read_output(file_path)

In [3]:
WIDTH = 0
HEIGHT = 1

In [4]:
upper_bound = sum(dims[:, HEIGHT])
k = 4 if not ROTATION else 8 # Number of constraints

In [5]:
prob = LpProblem("VLSI", LpMinimize)

In [6]:
L = LpVariable("L", 0, upper_bound, LpInteger)
pos_x = LpVariable.dicts("pos_x", range(n), 0, w, LpInteger)
pos_y = LpVariable.dicts("pos_y", range(n), 0, upper_bound, LpInteger)

In [7]:
M0 = M1 = w
M2 = M3 = upper_bound

In [8]:
pos_chosen = LpVariable.dict("pos_chosen", (range(n), range(n), range(k)), 0, 1, LpInteger)
rotated = LpVariable.dict("rotated", (range(n), range(2)), 0, 1, LpInteger)

In [9]:
prob += L

In [10]:
if ROTATION:
    for i in range(n):
        prob += L >= pos_y[i] + dims[i, HEIGHT]
        for j in range(i + 1, n):
            prob += pos_x[i] + dims[i, WIDTH] <= pos_x[j] + M0 * (pos_chosen[i, j, 0] + rotated[i, 0])
            prob += pos_x[j] + dims[j, WIDTH] <= pos_x[i] + M1 * (pos_chosen[i, j, 1] + rotated[i, 0])
            prob += pos_y[i] + dims[i, HEIGHT] <= pos_y[j] + M2 * (pos_chosen[i, j, 2] + rotated[i, 0])
            prob += pos_y[j] + dims[j, HEIGHT] <= pos_y[i] + M3 * (pos_chosen[i, j, 3] + rotated[i, 0])

            prob += pos_x[i] + dims[i, HEIGHT] <= pos_x[j] + M0 * (pos_chosen[i, j, 0] + rotated[i, 1])
            prob += pos_x[j] + dims[j, HEIGHT] <= pos_x[i] + M1 * (pos_chosen[i, j, 1] + rotated[i,1])
            prob += pos_y[i] + dims[i, WIDTH] <= pos_y[j] + M2 * (pos_chosen[i, j, 2] + rotated[i,1])
            prob += pos_y[j] + dims[j, WIDTH] <= pos_y[i] + M3 * (pos_chosen[i, j, 3] + rotated[i,1])
            
            prob += lpSum([pos_chosen[i, j, k] for k in range(k)]) <= 3
            prob += lpSum(rotated[i,k] for k in range(2)) == 1
            

        

In [11]:
if not ROTATION:
    for i in range(n):
        prob += L >= pos_y[i] + dims[i, HEIGHT]
        for j in range(i + 1, n):
            prob += pos_x[i] + dims[i, WIDTH] <= pos_x[j] + M0 * pos_chosen[i, j, 0]
            prob += pos_x[j] + dims[j, WIDTH] <= pos_x[i] + M1 * pos_chosen[i, j, 1]
            prob += pos_y[i] + dims[i, HEIGHT] <= pos_y[j] + M2 * pos_chosen[i, j, 2]
            prob += pos_y[j] + dims[j, HEIGHT] <= pos_y[i] + M3 * pos_chosen[i, j, 3]
            prob += lpSum([pos_chosen[i, j, k] for k in range(k)]) <= k - 1


In [12]:
status = prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/olestole/code/unibo/combinatorial/VLSIdesign/venv/lib/python3.7/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/vn/xw7vy4yj4p59wc8lpy7gnh200000gn/T/79105012a45c48cf83c0389d56a6444a-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/vn/xw7vy4yj4p59wc8lpy7gnh200000gn/T/79105012a45c48cf83c0389d56a6444a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 293 COLUMNS
At line 1997 RHS
At line 2286 BOUNDS
At line 2542 ENDATA
Problem MODEL has 288 rows, 255 columns and 1192 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 9 - 0.00 seconds
Cgl0004I processed model has 260 rows, 136 columns (136 integer (119 of which binary)) and 1024 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0045I 1 integer variables out of 136 objects (136 integer) have c

In [13]:
LpStatus[status]

'Optimal'

In [14]:
print(f"value of L: {value(L)}")
for i in range(n):
    print(value(pos_x[i]), value(pos_y[i]))

value of L: 9.0
6.0 6.0
2.0 3.0
9.0 0.0
6.0 3.0
6.0 0.0
3.0 0.0
0.0 0.0
9.0 6.0
